<a href="https://colab.research.google.com/github/AbrahamKong/temp/blob/main/temp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
# Import neccassary library

import pandas as pd
import numpy as np

In [69]:
# Read In text from files

data_dir = "/content/"
data_files_1 = [
    "q1.txt", "Document1.txt", "Document2.txt", "Document3.txt",
    "Document4.txt"
]

data_files_2 = [
    "q2.txt", "Document1.txt", "Document2.txt", "Document3.txt",
    "Document4.txt"
]

data1 = pd.Series()
data2 = pd.Series()
print("First set of Documents:")
for file in data_files_1:
    f = open(data_dir + file, 'r')
    file_contents = f.read()
    data1[file[:-4]] = file_contents
    f.close()
    print(file + ":" + data1[file[:-4]])
print("Second set of Documents:")
for file in data_files_2:
    f = open(data_dir + file, 'r')
    file_contents = f.read()
    data2[file[:-4]] = file_contents
    f.close()
    print(file + ":" + data2[file[:-4]])

First set of Documents:
q1.txt:say goodbye
Document1.txt:You say goodbye, I say hello
Document2.txt:You say stop, I say go
Document3.txt:Hello, hello, you say goodbye
Document4.txt:I say yes, you say no
Second set of Documents:
q2.txt:you hello
Document1.txt:You say goodbye, I say hello
Document2.txt:You say stop, I say go
Document3.txt:Hello, hello, you say goodbye
Document4.txt:I say yes, you say no


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app


In [70]:
# Count Word for each document

from collections import Counter

bag_of_words1 = (
    data1.
    str.lower().                  # convert all letters to lowercase
    str.replace("[^\w\s]", " ").  # replace non-alphanumeric characters by whitespace
    str.split()                   # split on whitespace
).apply(Counter)

bag_of_words2 = (
    data2.
    str.lower().                  # convert all letters to lowercase
    str.replace("[^\w\s]", " ").  # replace non-alphanumeric characters by whitespace
    str.split()                   # split on whitespace
).apply(Counter)

bag_of_words1
bag_of_words2

q2                                      {'you': 1, 'hello': 1}
Document1    {'you': 1, 'say': 2, 'goodbye': 1, 'i': 1, 'he...
Document2     {'you': 1, 'say': 2, 'stop': 1, 'i': 1, 'go': 1}
Document3       {'hello': 2, 'you': 1, 'say': 1, 'goodbye': 1}
Document4      {'i': 1, 'say': 2, 'yes': 1, 'you': 1, 'no': 1}
dtype: object

**TF-iDF**

In [71]:
# Set up TD

tf1 = pd.DataFrame(list(bag_of_words1))
tf1

,say,goodbye,you,i,hello,stop,go,yes,no
0,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
2,2,NaN,1.0,1.0,NaN,1.0,1.0,NaN,NaN
3,1,1.0,1.0,NaN,2.0,NaN,NaN,NaN,NaN
4,2,NaN,1.0,1.0,NaN,NaN,NaN,1.0,1.0


In [72]:
tf2 = pd.DataFrame(list(bag_of_words2))
tf2

,you,hello,say,goodbye,i,stop,go,yes,no
0,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1.0,2.0,1.0,1.0,NaN,NaN,NaN,NaN
2,1,NaN,2.0,NaN,1.0,1.0,1.0,NaN,NaN
3,1,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
4,1,NaN,2.0,NaN,1.0,NaN,NaN,1.0,1.0


In [73]:
# Fill N/A value with 0

tf1 = tf1.fillna(0)
tf1

,say,goodbye,you,i,hello,stop,go,yes,no
0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
2,2,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0
3,1,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0
4,2,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0


In [74]:
tf2 = tf2.fillna(0)
tf2

,you,hello,say,goodbye,i,stop,go,yes,no
0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0
2,1,0.0,2.0,0.0,1.0,1.0,1.0,0.0,0.0
3,1,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1,0.0,2.0,0.0,1.0,0.0,0.0,1.0,1.0


In [75]:
# Get document frequencies 

df1 = (tf1 > 0).sum(axis=0)
df1

say        5
goodbye    3
you        4
i          3
hello      2
stop       1
go         1
yes        1
no         1
dtype: int64

In [76]:
df2 = (tf2 > 0).sum(axis=0)
df2

you        5
hello      3
say        4
goodbye    2
i          3
stop       1
go         1
yes        1
no         1
dtype: int64

In [77]:
# Get IDFs


idf1 = np.log(len(tf1) / df1)
idf1

say        0.000000
goodbye    0.510826
you        0.223144
i          0.510826
hello      0.916291
stop       1.609438
go         1.609438
yes        1.609438
no         1.609438
dtype: float64

In [78]:
idf2 = np.log(len(tf2) / df2)
idf2

you        0.000000
hello      0.510826
say        0.223144
goodbye    0.916291
i          0.510826
stop       1.609438
go         1.609438
yes        1.609438
no         1.609438
dtype: float64

In [79]:
# Calculate TF-IDFs


tf_idf1 = tf1 * idf1
tf_idf1

,say,goodbye,you,i,hello,stop,go,yes,no
0,0.0,0.510826,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,0.510826,0.223144,0.510826,0.916291,0.000000,0.000000,0.000000,0.000000
2,0.0,0.000000,0.223144,0.510826,0.000000,1.609438,1.609438,0.000000,0.000000
3,0.0,0.510826,0.223144,0.000000,1.832581,0.000000,0.000000,0.000000,0.000000
4,0.0,0.000000,0.223144,0.510826,0.000000,0.000000,0.000000,1.609438,1.609438


In [80]:
tf_idf2 = tf2 * idf2
tf_idf2

,you,hello,say,goodbye,i,stop,go,yes,no
0,0.0,0.510826,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,0.510826,0.446287,0.916291,0.510826,0.000000,0.000000,0.000000,0.000000
2,0.0,0.000000,0.446287,0.000000,0.510826,1.609438,1.609438,0.000000,0.000000
3,0.0,1.021651,0.223144,0.916291,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.0,0.000000,0.446287,0.000000,0.510826,0.000000,0.000000,1.609438,1.609438


**Cisine Similarity**

In [81]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(norm=None) # Do not normalize.
vec.fit(data1) # This determines the vocabulary.
tf_idf_sparse1 = vec.transform(data1)

vec.fit(data2) # This determines the vocabulary.
tf_idf_sparse2 = vec.transform(data2)

In [82]:
# Calculate the numerator.
a1 = tf_idf_sparse1[0, :]
b1 = tf_idf_sparse1[2, :]
dot1 = a1.multiply(b1).sum()

# Calculate the terms in the denominator.
a_len1 = np.sqrt(a1.multiply(a1).sum())
b_len1 = np.sqrt(b1.multiply(b1).sum())

# Cosine similarity is their ratio.
dot1 / (a_len1 * b_len1)

0.30762584904158347

In [83]:
# Calculate the numerator.
a2 = tf_idf_sparse2[0, :]
b2 = tf_idf_sparse2[2, :]
dot2 = a2.multiply(b2).sum()

# Calculate the terms in the denominator.
a_len2 = np.sqrt(a2.multiply(a2).sum())
b_len2 = np.sqrt(b2.multiply(b2).sum())

# Cosine similarity is their ratio.
dot2 / (a_len2 * b_len2)

0.14773164862341073

In [84]:
# Calculate the numerators.
a1 = tf_idf_sparse1[0, :]
B1 = tf_idf_sparse1
dot1 = a1.multiply(B1).sum(axis=1)
dot1

matrix([[2.97533217],
        [3.97533217],
        [2.        ],
        [2.97533217],
        [2.        ]])

In [85]:
# Calculate the numerators.
a2 = tf_idf_sparse2[0, :]
B2 = tf_idf_sparse2
dot2 = a2.multiply(B2).sum(axis=1)
dot2

matrix([[2.97533217],
        [2.97533217],
        [1.        ],
        [4.95066434],
        [1.        ]])

In [86]:
# Calculate the denominators.
a_len1 = np.sqrt(a1.multiply(a1).sum())
b_len1 = np.sqrt(B1.multiply(B1).sum(axis=1))
print(a_len1)
b_len1

1.7249151196825583


matrix([[1.72491512],
        [3.19999435],
        [3.76911546],
        [3.97997562],
        [3.76911546]])

In [87]:
# Calculate the denominators.
a_len2 = np.sqrt(a2.multiply(a2).sum())
b_len2 = np.sqrt(B2.multiply(B2).sum(axis=1))
print(a_len2)
b_len2

1.7249151196825583


matrix([[1.72491512],
        [3.38136313],
        [3.92426861],
        [3.62849284],
        [3.92426861]])

In [88]:
# Calculate their ratio to obtain cosine similarities.
dot1 / (a_len1 * b_len1)

matrix([[1.        ],
        [0.72020558],
        [0.30762585],
        [0.43339841],
        [0.30762585]])

In [89]:
# Calculate their ratio to obtain cosine similarities.
dot2 / (a_len2 * b_len2)

matrix([[1.        ],
        [0.51012419],
        [0.14773165],
        [0.79098725],
        [0.14773165]])

In [90]:
cos_similarities1 = pd.DataFrame(dot1 / (a_len1 * b_len1))[0]
most_similar1 = cos_similarities1.sort_values(ascending=False)
most_similar1

0    1.000000
1    0.720206
3    0.433398
4    0.307626
2    0.307626
Name: 0, dtype: float64

In [91]:
cos_similarities2 = pd.DataFrame(dot2 / (a_len2 * b_len2))[0]
most_similar2 = cos_similarities2.sort_values(ascending=False)
most_similar2

0    1.000000
3    0.790987
1    0.510124
4    0.147732
2    0.147732
Name: 0, dtype: float64